In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as grs
from IPython.display import Image
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected = True)
#한글폰트설정
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_rows', 100)
#모델
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler
pd.options.display.float_format = '{:.5f}'.format
from lightgbm import LGBMClassifier, plot_importance
import lightgbm 
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from sklearn.linear_model import Lars,LassoLars,LassoLarsCV
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
df =pd.read_csv('한국전력거래소_5분단위 전력수급현황_20220407.csv',encoding='cp949')

In [3]:
df

,기준일시,공급능력(MW),현재수요(MW),최대예측수요(MW),공급예비력(MW),공급예비율(퍼센트),운영예비력(MW),운영예비율(퍼센트)
0,2012-06-01 00:00,64355.00000,51726.33000,59400,12628.67000,24.41439,12628.67000,24.41439
1,2012-06-01 00:05,64355.00000,51342.53000,60800,13012.47000,25.34442,13012.47000,25.34442
2,2012-06-01 00:10,64355.00000,51583.17000,60800,12771.83000,24.75969,12771.83000,24.75969
3,2012-06-01 00:15,64355.00000,51631.69000,60800,12723.31000,24.64245,12723.31000,24.64245
4,2012-06-01 00:20,64355.00000,51252.66000,60800,13102.35000,25.56423,13102.35000,25.56423
...,...,...,...,...,...,...,...,...
1035062,2022-04-07 23:35,82791.90000,63877.30000,69300,18914.70000,29.61100,10803.00000,16.91210
1035063,2022-04-07 23:40,82768.10000,63525.30000,69300,19242.80000,30.29160,10696.40000,16.83800
1035064,2022-04-07 23:45,82767.90000,63506.20000,69300,19261.70000,30.33040,10501.30000,16.53580
1035065,2022-04-07 23:50,82762.40000,62819.60000,69300,19942.80000,31.74610,10706.30000,17.04290


In [74]:
df_1=df[['기준일시','현재수요(MW)']]

In [79]:
df_2

,기준일시,현재수요(MW),next_1
0,2012-06-01 00:00,51726.33000,0
1,2012-06-01 00:05,51342.53000,0
2,2012-06-01 00:10,51583.17000,0
3,2012-06-01 00:15,51631.69000,0
4,2012-06-01 00:20,51252.66000,0
...,...,...,...
1035062,2022-04-07 23:35,63877.30000,0
1035063,2022-04-07 23:40,63525.30000,0
1035064,2022-04-07 23:45,63506.20000,0
1035065,2022-04-07 23:50,62819.60000,0


In [77]:
def forcast_next(df_1):
    df_1['next_1']=0
    for index in range(len(df_1)):
        try:
            df_1['next_1'][index] = df_1['현재수요(MW)'][index+72]
        except:continue
    return df_1

In [78]:
df_2 = forcast_next(df_2)

In [47]:
df_2 = df_1.iloc[100:-100]

In [48]:
#시계열 분해 잔차활용
def resid(df):
    date_resid=STL(df[['기준일시','현재수요(MW)']].set_index('기준일시'), period=12)
    df['resid']=date_resid.fit().resid.values
    return df

In [49]:
df_3 = resid(df_2)

In [63]:
df_3

,기준일시,현재수요(MW),next_1,resid
100,2012-06-01 08:30,56474.86000,60154.37000,-1351.07546
101,2012-06-01 08:35,57020.52000,60034.01000,-1008.58802
102,2012-06-01 08:40,57381.54000,59936.70000,-806.35205
103,2012-06-01 08:45,57688.42000,59868.69000,-563.68544
104,2012-06-01 08:50,57635.69000,59763.84000,-585.24074
...,...,...,...,...
1034962,2022-04-07 15:15,61313.70000,66904.20000,335.14652
1034963,2022-04-07 15:20,61500.80000,66622.40000,407.44174
1034964,2022-04-07 15:25,61762.60000,66606.10000,696.67833
1034965,2022-04-07 15:30,62271.70000,66192.80000,987.96747


In [50]:
def split_train_and_test(df, date):
    """
    Dataframe에서 train_df, test_df로 나눠주는 함수
    df : 시계열 데이터 프레임
    date : 기준점 날짜
    """
    train = df[df['기준일시'] < date]
    test = df[df['기준일시'] >= date]
    del train['기준일시']
    del test['기준일시']
    y_train = train.pop(f'next_1')
    x_train=train.copy()
    y_test = test.pop(f'next_1')
    x_test=test.copy()
    return x_train,y_train,x_test,y_test

In [56]:
x_train,y_train,x_test,y_test= split_train_and_test(df_3,'2021-01-07 23:35')

In [64]:
x_train

,현재수요(MW),resid
100,56474.86000,-1351.07546
101,57020.52000,-1008.58802
102,57381.54000,-806.35205
103,57688.42000,-563.68544
104,57635.69000,-585.24074
...,...,...
904423,76147.70000,712.30307
904424,76051.00000,510.90254
904425,76333.10000,682.19437
904426,76172.60000,674.42955


In [57]:
models = []
models.append(('lgbm',LGBMRegressor(num_leaves=50,
    max_depth=30,
    metric='mape',
    learning_rate=0.01,
    n_estimators=500,
    max_bin = 59,)))
# models.append(('xgboost',XGBRegressor(     learning_rate = 0.01,
#     max_depth = 11,
#     n_estimators = 550,)))
# models.append(('rf',RandomForestRegressor( n_estimators = 355,
#     max_depth = 10,)))
models.append(('linear',LinearRegression()))
models.append(('lasso',Lasso()))
# models.append(('SVR', SVR(gamma='auto'))) # kernel = linear
#models.append(('SVR',Lasso()))
#models.append(('llars',LassoLars()))
result=[]
names=[]
print('전력 예측')
for name,model in models:
    tscv = TimeSeriesSplit(n_splits=5)
    cv_result_mape = cross_val_score(model,x_train,y_train,cv=tscv,scoring='neg_mean_absolute_percentage_error')
    #cv_result_mae = cross_val_score(model,x_train,y_train,cv=tscv,scoring='neg_mean_absolute_error')
    #cv_mae_mean=cv_result_mae.mean()
    print(cv_result_mape)
    cv_mape_mean=cv_result_mape.mean()
    cv_std = cv_result_mape.std()
    result.append(cv_result_mape)
    names.append(name)
    print(f'{name} :  mape {cv_mape_mean} ({cv_std})')

전력 예측
[-0.06639415 -0.08209816 -0.07650862 -0.07701106 -0.08219357]
lgbm :  mape -0.0768411121751714 (0.005754369697346665)
[-0.06719786 -0.08428453 -0.07751221 -0.07810641 -0.08409718]
linear :  mape -0.0782396361062635 (0.006218027497321533)
[-0.06719786 -0.08428453 -0.07751221 -0.07810641 -0.08409718]
lasso :  mape -0.07823963728490305 (0.006218027391486373)


In [59]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()
lgb.fit(x_train,y_train)
predict = lgb.predict(x_test)

In [60]:
predict

array([69594.86183744, 70752.86998513, 73067.76547256, ...,
       58647.72584302, 58715.80564791, 59041.46137409])

In [61]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(y_test,predict)

0.08493804508777635